# Chile Consolidación de Datos territoriales

In [1]:
import pandas as pd
import json
import unittest

## Extración

### Listado y Nombre de Regiones según la Ley 21.074 del 2018

In [2]:
ley_regiones_df = pd.read_csv("sources/ley-regiones.csv", dtype=str)
ley_regiones_df

,código,nombre
0,15,Región de Arica y Parinacota
1,01,Región de Tarapacá
2,02,Región de Antofagasta
3,03,Región de Atacama
4,04,Región de Coquimbo
5,05,Región de Valparaíso
6,13,Región Metropolitana de Santiago
7,06,Región del Libertador General Bernardo O'Higgins
8,07,Región del Maule
9,16,Región de Ñuble


### Consolidado de Regiones, Provincias y Comunas hasta el 2018

In [3]:
observatoriologistico_df = pd.read_csv("sources/observatoriologistico.csv", dtype=str)
observatoriologistico_df

,Código Región,Nombre Región,Abreviatura Región,Código Provincia,Nombre Provincia,Código Comuna 2018,Nombre Comuna
0,01,Tarapacá,TPCA,011,Iquique,01101,Iquique
1,01,Tarapacá,TPCA,011,Iquique,01107,Alto Hospicio
2,01,Tarapacá,TPCA,014,Tamarugal,01401,Pozo Almonte
3,01,Tarapacá,TPCA,014,Tamarugal,01402,Camiña
4,01,Tarapacá,TPCA,014,Tamarugal,01403,Colchane
...,...,...,...,...,...,...,...
341,16,Ñuble,NUBLE,163,Punilla,16301,San Carlos
342,16,Ñuble,NUBLE,163,Punilla,16302,Coihueco
343,16,Ñuble,NUBLE,163,Punilla,16303,Ñiquén
344,16,Ñuble,NUBLE,163,Punilla,16304,San Fabián


### Estandarización Internacional Territorial Chile

In [4]:

iso_df = pd.read_csv("sources/iso.csv")
iso_df

,Subdivision category,3166-2 code,Subdivision name,Local variant,Language code,Romanization system,Parent subdivision
0,region,CL-AI*,Aisén del General Carlos Ibañez del Campo,"Aysén, Aisén",es,NaN,NaN
1,region,CL-AN*,Antofagasta,NaN,es,NaN,NaN
2,region,CL-AP*,Arica y Parinacota,NaN,es,NaN,NaN
3,region,CL-AT*,Atacama,NaN,es,NaN,NaN
4,region,CL-BI*,Biobío,NaN,es,NaN,NaN
5,region,CL-CO*,Coquimbo,NaN,es,NaN,NaN
6,region,CL-AR*,La Araucanía,NaN,es,NaN,NaN
7,region,CL-LI*,Libertador General Bernardo O'Higgins,O'Higgins,es,NaN,NaN
8,region,CL-LL*,Los Lagos,NaN,es,NaN,NaN
9,region,CL-LR*,Los Ríos,NaN,es,NaN,NaN


## Transformaciones

In [5]:
def apply_replacements(text, replacements):
    for wrong, correct in replacements.items():
        text = text.replace(wrong, correct)
    return text

replacements = {
    "Aisén del General Carlos Ibañez del Campo": "Aysén del General Carlos Ibáñez del Campo",
    "Magallanes": "Magallanes y de la Antártica Chilena",
    "Región Metropolitana de Santiago": "Metropolitana de Santiago",
}
iso_df["corrected_name"] = iso_df["Subdivision name"].apply(
    lambda x: apply_replacements(x, replacements)
)

# Preprocesar iso_df para hacer la correspondencia más directa y limpiar el código ISO
iso_df["clean_iso_code"] = iso_df["3166-2 code"].str.replace("*", "")

# Unir los dataframes en los nombres corregidos y necesarios
merged_df = observatoriologistico_df.merge(
    iso_df[["corrected_name", "clean_iso_code"]],
    left_on="Nombre Región",
    right_on="corrected_name",
    how="left",
)
merged_df = merged_df.merge(
    ley_regiones_df[["código", "nombre"]],
    left_on="Código Región",
    right_on="código",
    how="left",
)
merged_df

,Código Región,Nombre Región,Abreviatura Región,Código Provincia,Nombre Provincia,Código Comuna 2018,Nombre Comuna,corrected_name,clean_iso_code,código,nombre
0,01,Tarapacá,TPCA,011,Iquique,01101,Iquique,Tarapacá,CL-TA,01,Región de Tarapacá
1,01,Tarapacá,TPCA,011,Iquique,01107,Alto Hospicio,Tarapacá,CL-TA,01,Región de Tarapacá
2,01,Tarapacá,TPCA,014,Tamarugal,01401,Pozo Almonte,Tarapacá,CL-TA,01,Región de Tarapacá
3,01,Tarapacá,TPCA,014,Tamarugal,01402,Camiña,Tarapacá,CL-TA,01,Región de Tarapacá
4,01,Tarapacá,TPCA,014,Tamarugal,01403,Colchane,Tarapacá,CL-TA,01,Región de Tarapacá
...,...,...,...,...,...,...,...,...,...,...,...
341,16,Ñuble,NUBLE,163,Punilla,16301,San Carlos,Ñuble,CL-NB,16,Región de Ñuble
342,16,Ñuble,NUBLE,163,Punilla,16302,Coihueco,Ñuble,CL-NB,16,Región de Ñuble
343,16,Ñuble,NUBLE,163,Punilla,16303,Ñiquén,Ñuble,CL-NB,16,Región de Ñuble
344,16,Ñuble,NUBLE,163,Punilla,16304,San Fabián,Ñuble,CL-NB,16,Región de Ñuble


## Carga

### CSV
#### Construcción de Regiones CSV

In [6]:
# Seleccionar y renombrar las columnas necesarias para el DataFrame final
regions_df = merged_df[
    ["Código Región", "nombre", "Nombre Región", "Abreviatura Región", "clean_iso_code"]
]
regions_df.columns = ["id", "name", "short_name", "abbreviation", "iso_code"]
# Pylance:
regions_df = regions_df.drop_duplicates(subset=["id"], keep="last")

# Guardar a CSV y mostrar el DataFrame
regions_df.to_csv("csv/regions.csv", index=False)
regions_df

,id,name,short_name,abbreviation,iso_code
6,01,Región de Tarapacá,Tarapacá,TPCA,CL-TA
15,02,Región de Antofagasta,Antofagasta,ANTOF,CL-AN
24,03,Región de Atacama,Atacama,ATCMA,CL-AT
39,04,Región de Coquimbo,Coquimbo,COQ,CL-CO
77,05,Región de Valparaíso,Valparaíso,VALPO,CL-VS
110,06,Región del Libertador General Bernardo O'Higgins,Libertador General Bernardo O'Higgins,LGBO,CL-LI
140,07,Región del Maule,Maule,MAULE,CL-ML
173,08,Región del Biobío,Biobío,BBIO,CL-BI
205,09,Región de La Araucanía,La Araucanía,ARAUC,CL-AR
235,10,Región de Los Lagos,Los Lagos,LAGOS,CL-LL


#### Construcción de Provincias CSV

In [7]:
provinces_df = observatoriologistico_df[
    ["Código Provincia", "Nombre Provincia", "Código Región"]
].copy()
provinces_df.columns = ["id", "name", "region_id"]

provinces_df = provinces_df.drop_duplicates(subset=["id"], keep="last")

provinces_df.to_csv("csv/provinces.csv", index=False)
provinces_df

,id,name,region_id
1,011,Iquique,01
6,014,Tamarugal,01
10,021,Antofagasta,02
13,022,El Loa,02
15,023,Tocopilla,02
18,031,Copiapó,03
20,032,Chañaral,03
24,033,Huasco,03
30,041,Elqui,04
34,042,Choapa,04


#### Construcción de Comunas CSV

In [8]:
communes_df = observatoriologistico_df[
    ["Código Comuna 2018", "Nombre Comuna", "Código Provincia"]
].copy()
communes_df.columns = ["id", "name", "province_id"]

communes_df.to_csv("csv/communes.csv", index=False)
communes_df

,id,name,province_id
0,01101,Iquique,011
1,01107,Alto Hospicio,011
2,01401,Pozo Almonte,014
3,01402,Camiña,014
4,01403,Colchane,014
...,...,...,...
341,16301,San Carlos,163
342,16302,Coihueco,163
343,16303,Ñiquén,163
344,16304,San Fabián,163


## JSON

#### Datos Territoriales como objeto JSON

In [9]:
# Agrupar comunas por provincia_id y construir el diccionario de comunas
communes_grouped = communes_df.groupby("province_id")
provinces_df["communes"] = provinces_df["id"].apply(
    lambda x: {
        row["id"]: {"id": row["id"], "name": row["name"]}
        for _, row in communes_grouped.get_group(x).iterrows()
        if x in communes_grouped.groups
    }
)

# Agrupar provincias por region_id y construir el diccionario de provincias
provinces_grouped = provinces_df.groupby("region_id")
regions_df["provinces"] = regions_df["id"].apply(
    lambda x: {
        row["id"]: {"id": row["id"], "name": row["name"], "communes": row["communes"]}
        for _, row in provinces_grouped.get_group(x).iterrows()
        if x in provinces_grouped.groups
    }
)

# Construir el diccionario final para el país, asumiendo que el país es Chile
cl_country = {
    "country": "Chile",
    "regions": {
        row["id"]: {
            "id": row["id"],
            "name": row["name"],
            "short_name": row["short_name"],
            "abbreviation": row["abbreviation"],
            "iso_code": row["iso_code"],
            "provinces": row["provinces"],
        }
        for _, row in regions_df.iterrows()
    },
}

# Convertir el diccionario a JSON
json_data = json.dumps(cl_country, ensure_ascii=False, indent=2)

# Guardar JSON en un archivo
with open("json/data.json", "w", encoding="utf-8") as f:
    f.write(json_data)

# Imprimir el JSON para verificar
print(json_data)

{
  "country": "Chile",
  "regions": {
    "01": {
      "id": "01",
      "name": "Región de Tarapacá",
      "short_name": "Tarapacá",
      "abbreviation": "TPCA",
      "iso_code": "CL-TA",
      "provinces": {
        "011": {
          "id": "011",
          "name": "Iquique",
          "communes": {
            "01101": {
              "id": "01101",
              "name": "Iquique"
            },
            "01107": {
              "id": "01107",
              "name": "Alto Hospicio"
            }
          }
        },
        "014": {
          "id": "014",
          "name": "Tamarugal",
          "communes": {
            "01401": {
              "id": "01401",
              "name": "Pozo Almonte"
            },
            "01402": {
              "id": "01402",
              "name": "Camiña"
            },
            "01403": {
              "id": "01403",
              "name": "Colchane"
            },
            "01404": {
              "id": "01404",
              "na

### Datos Territoriales como arreglo JSON

In [19]:
communes_grouped = communes_df.groupby("province_id")
provinces_df["communes"] = provinces_df["id"].apply(
    lambda x: [
        {"id": row["id"], "name": row["name"]}
        for _, row in communes_grouped.get_group(x).iterrows()
        if x in communes_grouped.groups
    ]
)

provinces_grouped = provinces_df.groupby("region_id")
regions_df["provinces"] = regions_df["id"].apply(
    lambda x: [
        {"id": row["id"],"name": row["name"],"communes": row["communes"]}
        for _, row in provinces_grouped.get_group(x).iterrows()
        if x in provinces_grouped.groups
    ]
)

regions = {
    "country": "Chile",
    "regions": [
        {
            "id": row["id"],
            "name": row["name"],
            "short_name": row["short_name"],
            "abbreviation": row["abbreviation"],
            "iso_code": row["iso_code"],
            "provinces": row["provinces"],
        }
        for _, row in regions_df.iterrows()
    ],
}
regions



{'country': 'Chile',
 'regions': [{'id': '01',
   'name': 'Región de Tarapacá',
   'short_name': 'Tarapacá',
   'abbreviation': 'TPCA',
   'iso_code': 'CL-TA',
   'provinces': [{'id': '011',
     'name': 'Iquique',
     'communes': [{'id': '01101', 'name': 'Iquique'},
      {'id': '01107', 'name': 'Alto Hospicio'}]},
    {'id': '014',
     'name': 'Tamarugal',
     'communes': [{'id': '01401', 'name': 'Pozo Almonte'},
      {'id': '01402', 'name': 'Camiña'},
      {'id': '01403', 'name': 'Colchane'},
      {'id': '01404', 'name': 'Huara'},
      {'id': '01405', 'name': 'Pica'}]}]},
  {'id': '02',
   'name': 'Región de Antofagasta',
   'short_name': 'Antofagasta',
   'abbreviation': 'ANTOF',
   'iso_code': 'CL-AN',
   'provinces': [{'id': '021',
     'name': 'Antofagasta',
     'communes': [{'id': '02101', 'name': 'Antofagasta'},
      {'id': '02102', 'name': 'Mejillones'},
      {'id': '02103', 'name': 'Sierra Gorda'},
      {'id': '02104', 'name': 'Taltal'}]},
    {'id': '022',
     '

## Construcción de Región SQL

In [ ]:
# Definir el modelo de la tabla
table_model = """
CREATE TABLE region (
  id VARCHAR(2) PRIMARY KEY,
  name VARCHAR(255),
  short_name VARCHAR(255),
  abbreviation VARCHAR(10),
  iso_code VARCHAR(10)
);
"""

# Generar las sentencias INSERT
insert_statements = ""
for _, row in regions_df.iterrows():
    insert_statements += f"INSERT INTO region (id, name, short_name, abbreviation, iso_code) VALUES ('{row['id']}', '{row['name'].replace("'","''")}', '{row['short_name'].replace("'","''")}', '{row['abbreviation']}', '{row['iso_code']}');\n"

# Combinar el modelo de la tabla y las sentencias INSERT
sql_script = table_model + "\n" + insert_statements

# Guardar el script SQL en un archivo
with open("sql/regions.sql", "w", encoding="utf-8") as f:
    f.write(sql_script)

# Imprimir el script SQL para verificar
print(sql_script)

In [ ]:
# Definir el modelo de la tabla
provinces_table_model = """
CREATE TABLE province (
  id VARCHAR(3) PRIMARY KEY,
  name VARCHAR(255),
  region_id VARCHAR(2),
  FOREIGN KEY (region_id) REFERENCES region(id)
);
"""

# Generar las sentencias INSERT
provinces_insert_statements = ""
for _, row in provinces_df.iterrows():
    provinces_insert_statements += f"INSERT INTO province (id, name, region_id) VALUES ('{row['id']}', '{row['name'].replace("'","''")}', '{row['region_id']}');\n"

# Combinar el modelo de la tabla y las sentencias INSERT
provinces_sql_script = provinces_table_model + "\n" + provinces_insert_statements

# Guardar el script SQL en un archivo
with open("sql/provinces.sql", "w", encoding="utf-8") as f:
    f.write(provinces_sql_script)

# Imprimir el script SQL para verificar
print(provinces_sql_script)

In [ ]:
# Definir el modelo de la tabla
communes_table_model = """
CREATE TABLE commune (
  id VARCHAR(5) PRIMARY KEY,
  name VARCHAR(255),
  province_id VARCHAR(3),
  FOREIGN KEY (province_id) REFERENCES province(id)
);
"""

# Generar las sentencias INSERT
communes_insert_statements = ""
for _, row in communes_df.iterrows():
    communes_insert_statements += f"INSERT INTO commune (id, name, province_id) VALUES ('{row['id']}', '{row['name'].replace("'","''")}', '{row['province_id']}');\n"

# Combinar el modelo de la tabla y las sentencias INSERT
communes_sql_script = communes_table_model + "\n" + communes_insert_statements

# Guardar el script SQL en un archivo
with open("sql/communes.sql", "w", encoding="utf-8") as f:
    f.write(communes_sql_script)

# Imprimir el script SQL para verificar
print(communes_sql_script)